In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math    
import folium
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
import matplotlib.image as mpimg
import glob, os
from pathlib import Path
from IPython.core.display import display,HTML
pd.set_option('display.max_colwidth', None)

In [2]:
from src.cluster import *
from src.colors import *
from src.map import *
from src.compare import *

In [3]:
labels = pd.read_csv('datasets/mikey-devon-labels_validation-221109.csv')

In [4]:
#create labels geo dataframe
points = labels.apply(lambda row: Point(row.lng, row.lat), axis=1)
labels_geo = gpd.GeoDataFrame(labels, geometry=points)
labels_geo.crs = {'init': 'epsg:4326'}
labels_geo

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,Unnamed: 14,Unnamed: 15,Unnamed: 16,geometry
0,Devon Snyder,3,480,University District,Obstacle,47.661041,-122.320709,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:36.787-07,t,3.0,f,pole,NaN,NaN,NaN,NaN,POINT (-122.32071 47.66104)
1,Devon Snyder,4,480,University District,Obstacle,47.661026,-122.320702,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:52.367-07,t,2.0,f,pole,NaN,NaN,NaN,NaN,POINT (-122.32070 47.66103)
2,Devon Snyder,5,480,University District,SurfaceProblem,47.661060,-122.320709,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:01:21.608-07,t,2.0,f,bumpy,NaN,NaN,NaN,NaN,POINT (-122.32071 47.66106)
3,Devon Snyder,8,480,University District,Occlusion,47.661106,-122.320724,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:02:15.133-07,NaN,NaN,f,NaN,NaN,NaN,NaN,NaN,POINT (-122.32072 47.66111)
4,Devon Snyder,9,480,University District,Obstacle,47.661098,-122.320663,O0VZ-MF6Nrmjhkb2nCRhCw,2022-05-10 14:02:29.636-07,t,3.0,f,sign,temporary due to construction,NaN,NaN,NaN,POINT (-122.32066 47.66110)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020,mikey,2493,810,Windermere,SurfaceProblem,47.672043,-122.267929,Z6BJD4bSdDiMxwMdls5oNw,2022-07-06 14:12:40.653-07,NaN,1.0,f,uneven/slanted,NaN,NaN,NaN,NaN,POINT (-122.26793 47.67204)
2021,mikey,2494,716,Windermere,Obstacle,47.671928,-122.268097,sOyfUL3XT8ETE-9XfeaXvQ,2022-07-06 14:14:34.691-07,NaN,1.0,f,vegetation,NaN,NaN,NaN,NaN,POINT (-122.26810 47.67193)
2022,mikey,2495,715,Windermere,NoSidewalk,47.672016,-122.268433,xbqjQfJSHA1O90JnBHQJGw,2022-07-06 14:14:48.626-07,NaN,3.0,f,street has a sidewalk,NaN,NaN,NaN,NaN,POINT (-122.26843 47.67202)
2023,mikey,2496,716,Windermere,SurfaceProblem,47.671883,-122.268181,xbqjQfJSHA1O90JnBHQJGw,2022-07-06 14:14:55.957-07,NaN,1.0,f,cracks,NaN,NaN,NaN,NaN,POINT (-122.26818 47.67188)


In [5]:
# Seperate Seattle and Chicago data
chicago = labels_geo.loc[labels_geo['lng'] > -105]
seattle = labels_geo.loc[labels_geo['lng'] < -105]

In [7]:
#map_clusters(chicago,0)

In [8]:
#get curb ramp cluster dataset for chicago 
chicago_curb_ramp = cluster_label_type_at_index(chicago,0)[2]
#get problem cluster dataset for chicago 
chicago_problem = cluster_label_type_at_index(chicago,2)[2]

In [9]:
# locate folder with images
chicago_folder ="chicago_with_icons/"

In [10]:
def append_img_url(df, folder):
    df['img_url'] = df.apply(lambda x: 'img/'+folder + "label_id_"+str(x.label_id)+".jpg", axis=1)
    return df

In [11]:
append_img_url(chicago_curb_ramp, chicago_folder)

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,Unnamed: 14,Unnamed: 15,Unnamed: 16,geometry,cluster_id,img_url
76,Devon Snyder,94,447,Skokie,CurbRamp,42.022606,-87.728157,rcjtiNG8gkNtkifZDFj2NQ,2022-05-10 14:56:14.398-07,t,4.0,f,not level with street,not enough landing space,narrow,NaN,NaN,POINT (-87.72816 42.02261),2,img/chicago_with_icons/label_id_94.jpg
77,Devon Snyder,96,447,Skokie,CurbRamp,42.022720,-87.728149,rcjtiNG8gkNtkifZDFj2NQ,2022-05-10 14:56:58.603-07,t,3.0,f,not enough landing space,surface problem,NaN,NaN,NaN,POINT (-87.72815 42.02272),1,img/chicago_with_icons/label_id_96.jpg
78,Devon Snyder,97,437,Skokie,CurbRamp,42.022717,-87.728432,_KrmBjHlebAncoZc40tsSg,2022-05-10 14:57:43.381-07,t,3.0,f,not enough landing space,points into traffic,NaN,NaN,NaN,POINT (-87.72843 42.02272),15,img/chicago_with_icons/label_id_97.jpg
79,Devon Snyder,98,437,Skokie,CurbRamp,42.022621,-87.728439,_KrmBjHlebAncoZc40tsSg,2022-05-10 14:58:11.132-07,t,3.0,f,not enough landing space,NaN,NaN,NaN,NaN,POINT (-87.72844 42.02262),16,img/chicago_with_icons/label_id_98.jpg
82,Devon Snyder,102,447,Skokie,CurbRamp,42.023483,-87.728134,BMhU3MUjwcgFnO2ax-6TxQ,2022-05-10 15:00:54.488-07,t,3.0,f,surface problem,points into traffic,NaN,NaN,NaN,POINT (-87.72813 42.02348),3,img/chicago_with_icons/label_id_102.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1914,mikey,2385,447,Skokie,CurbRamp,42.024353,-87.728111,T14zUxLr9qqMbsGQ_9Jqqg,2022-07-05 12:41:14.317-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72811 42.02435),5,img/chicago_with_icons/label_id_2385.jpg
1916,mikey,2387,447,Skokie,CurbRamp,42.023594,-87.728142,tagGeC6m-TMnzcfHLPX6Hg,2022-07-05 12:41:55.266-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72814 42.02359),4,img/chicago_with_icons/label_id_2387.jpg
1917,mikey,2388,447,Skokie,CurbRamp,42.023453,-87.728127,TdCToHEwW86oaUfwfNNDaw,2022-07-05 12:42:00.438-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72813 42.02345),3,img/chicago_with_icons/label_id_2388.jpg
1918,mikey,2389,447,Skokie,CurbRamp,42.022594,-87.728149,rcjtiNG8gkNtkifZDFj2NQ,2022-07-05 12:42:51.976-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72815 42.02259),2,img/chicago_with_icons/label_id_2389.jpg


In [12]:
chicago_curb_ramp

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,Unnamed: 14,Unnamed: 15,Unnamed: 16,geometry,cluster_id,img_url
76,Devon Snyder,94,447,Skokie,CurbRamp,42.022606,-87.728157,rcjtiNG8gkNtkifZDFj2NQ,2022-05-10 14:56:14.398-07,t,4.0,f,not level with street,not enough landing space,narrow,NaN,NaN,POINT (-87.72816 42.02261),2,img/chicago_with_icons/label_id_94.jpg
77,Devon Snyder,96,447,Skokie,CurbRamp,42.022720,-87.728149,rcjtiNG8gkNtkifZDFj2NQ,2022-05-10 14:56:58.603-07,t,3.0,f,not enough landing space,surface problem,NaN,NaN,NaN,POINT (-87.72815 42.02272),1,img/chicago_with_icons/label_id_96.jpg
78,Devon Snyder,97,437,Skokie,CurbRamp,42.022717,-87.728432,_KrmBjHlebAncoZc40tsSg,2022-05-10 14:57:43.381-07,t,3.0,f,not enough landing space,points into traffic,NaN,NaN,NaN,POINT (-87.72843 42.02272),15,img/chicago_with_icons/label_id_97.jpg
79,Devon Snyder,98,437,Skokie,CurbRamp,42.022621,-87.728439,_KrmBjHlebAncoZc40tsSg,2022-05-10 14:58:11.132-07,t,3.0,f,not enough landing space,NaN,NaN,NaN,NaN,POINT (-87.72844 42.02262),16,img/chicago_with_icons/label_id_98.jpg
82,Devon Snyder,102,447,Skokie,CurbRamp,42.023483,-87.728134,BMhU3MUjwcgFnO2ax-6TxQ,2022-05-10 15:00:54.488-07,t,3.0,f,surface problem,points into traffic,NaN,NaN,NaN,POINT (-87.72813 42.02348),3,img/chicago_with_icons/label_id_102.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1914,mikey,2385,447,Skokie,CurbRamp,42.024353,-87.728111,T14zUxLr9qqMbsGQ_9Jqqg,2022-07-05 12:41:14.317-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72811 42.02435),5,img/chicago_with_icons/label_id_2385.jpg
1916,mikey,2387,447,Skokie,CurbRamp,42.023594,-87.728142,tagGeC6m-TMnzcfHLPX6Hg,2022-07-05 12:41:55.266-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72814 42.02359),4,img/chicago_with_icons/label_id_2387.jpg
1917,mikey,2388,447,Skokie,CurbRamp,42.023453,-87.728127,TdCToHEwW86oaUfwfNNDaw,2022-07-05 12:42:00.438-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72813 42.02345),3,img/chicago_with_icons/label_id_2388.jpg
1918,mikey,2389,447,Skokie,CurbRamp,42.022594,-87.728149,rcjtiNG8gkNtkifZDFj2NQ,2022-07-05 12:42:51.976-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72815 42.02259),2,img/chicago_with_icons/label_id_2389.jpg


In [13]:
chicago_curb_ramp["clustered"] = chicago_curb_ramp.duplicated(subset="cluster_id", keep=False)

In [14]:
chicago_curb_ramp

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,Unnamed: 14,Unnamed: 15,Unnamed: 16,geometry,cluster_id,img_url,clustered
76,Devon Snyder,94,447,Skokie,CurbRamp,42.022606,-87.728157,rcjtiNG8gkNtkifZDFj2NQ,2022-05-10 14:56:14.398-07,t,4.0,f,not level with street,not enough landing space,narrow,NaN,NaN,POINT (-87.72816 42.02261),2,img/chicago_with_icons/label_id_94.jpg,True
77,Devon Snyder,96,447,Skokie,CurbRamp,42.022720,-87.728149,rcjtiNG8gkNtkifZDFj2NQ,2022-05-10 14:56:58.603-07,t,3.0,f,not enough landing space,surface problem,NaN,NaN,NaN,POINT (-87.72815 42.02272),1,img/chicago_with_icons/label_id_96.jpg,True
78,Devon Snyder,97,437,Skokie,CurbRamp,42.022717,-87.728432,_KrmBjHlebAncoZc40tsSg,2022-05-10 14:57:43.381-07,t,3.0,f,not enough landing space,points into traffic,NaN,NaN,NaN,POINT (-87.72843 42.02272),15,img/chicago_with_icons/label_id_97.jpg,True
79,Devon Snyder,98,437,Skokie,CurbRamp,42.022621,-87.728439,_KrmBjHlebAncoZc40tsSg,2022-05-10 14:58:11.132-07,t,3.0,f,not enough landing space,NaN,NaN,NaN,NaN,POINT (-87.72844 42.02262),16,img/chicago_with_icons/label_id_98.jpg,True
82,Devon Snyder,102,447,Skokie,CurbRamp,42.023483,-87.728134,BMhU3MUjwcgFnO2ax-6TxQ,2022-05-10 15:00:54.488-07,t,3.0,f,surface problem,points into traffic,NaN,NaN,NaN,POINT (-87.72813 42.02348),3,img/chicago_with_icons/label_id_102.jpg,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1914,mikey,2385,447,Skokie,CurbRamp,42.024353,-87.728111,T14zUxLr9qqMbsGQ_9Jqqg,2022-07-05 12:41:14.317-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72811 42.02435),5,img/chicago_with_icons/label_id_2385.jpg,True
1916,mikey,2387,447,Skokie,CurbRamp,42.023594,-87.728142,tagGeC6m-TMnzcfHLPX6Hg,2022-07-05 12:41:55.266-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72814 42.02359),4,img/chicago_with_icons/label_id_2387.jpg,True
1917,mikey,2388,447,Skokie,CurbRamp,42.023453,-87.728127,TdCToHEwW86oaUfwfNNDaw,2022-07-05 12:42:00.438-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72813 42.02345),3,img/chicago_with_icons/label_id_2388.jpg,True
1918,mikey,2389,447,Skokie,CurbRamp,42.022594,-87.728149,rcjtiNG8gkNtkifZDFj2NQ,2022-07-05 12:42:51.976-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72815 42.02259),2,img/chicago_with_icons/label_id_2389.jpg,True


In [18]:
chicago_curb_ramp.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/chicago_curb_ramp.json', driver="GeoJSON") 

In [34]:
append_img_url(chicago_problem, chicago_folder)

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
92,Devon Snyder,95,447,Skokie,SurfaceProblem,42.022564,-87.728165,rcjtiNG8gkNtkifZDFj2NQ,2022-05-10 14:56:32.47-07,t,1.0,f,grass,NaN,POINT (-87.72816 42.02256),168,img/chicago_with_icons/label_id_95.jpg
96,Devon Snyder,99,447,Skokie,SurfaceProblem,42.022999,-87.728149,2vMz67hXm2ZGNcKHbyoR8g,2022-05-10 14:59:05.467-07,t,1.0,f,grass,NaN,POINT (-87.72815 42.02300),67,img/chicago_with_icons/label_id_99.jpg
97,Devon Snyder,100,447,Skokie,SurfaceProblem,42.023033,-87.728149,2vMz67hXm2ZGNcKHbyoR8g,2022-05-10 14:59:10.286-07,t,1.0,f,NaN,NaN,POINT (-87.72815 42.02303),65,img/chicago_with_icons/label_id_100.jpg
99,Devon Snyder,103,447,Skokie,SurfaceProblem,42.023464,-87.728004,BMhU3MUjwcgFnO2ax-6TxQ,2022-05-10 15:01:27.068-07,t,2.0,f,"cracks,grass",NaN,POINT (-87.72800 42.02346),71,img/chicago_with_icons/label_id_103.jpg
101,Devon Snyder,105,447,Skokie,SurfaceProblem,42.023663,-87.728111,spnkMkInfnfp6Se2o7-t9Q,2022-05-10 15:02:43.061-07,t,1.0,f,"grass,cracks",NaN,POINT (-87.72811 42.02366),73,img/chicago_with_icons/label_id_105.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,mikey,2368,437,Skokie,SurfaceProblem,42.023743,-87.728401,UUSs_oAVBV5cqnoLKsUnlQ,2022-07-05 12:25:19.838-07,NaN,1.0,f,NaN,NaN,POINT (-87.72840 42.02374),7,img/chicago_with_icons/label_id_2368.jpg
2250,mikey,2369,437,Skokie,SurfaceProblem,42.023582,-87.728416,Ue0KqJfGftuJWhdBCneCkQ,2022-07-05 12:25:31.351-07,NaN,1.0,f,uneven/slanted,NaN,POINT (-87.72842 42.02358),6,img/chicago_with_icons/label_id_2369.jpg
2259,mikey,2378,404,Skokie,SurfaceProblem,42.025799,-87.728088,jfd17OkETDGKKw-I1zK6Jw,2022-07-05 12:38:58.8-07,NaN,1.0,f,bumpy,NaN,POINT (-87.72809 42.02580),5,img/chicago_with_icons/label_id_2378.jpg
2261,mikey,2380,404,Skokie,Obstacle,42.025677,-87.728088,Y-iz2Xxh7q_fAX_UH6mmVA,2022-07-05 12:39:07.658-07,NaN,1.0,t,NaN,NaN,POINT (-87.72809 42.02568),3,img/chicago_with_icons/label_id_2380.jpg


In [15]:
#get center location of the geodf
chicago_curb_ramp.dissolve().centroid

0    POINT (-87.76274 41.93642)
dtype: geometry

In [16]:
#get curb ramp cluster dataset for seattle 
seattle_curb_ramp = cluster_label_type_at_index(seattle,0)[2]
#get problem cluster dataset for seattle 
seattle_problem = cluster_label_type_at_index(seattle,2)[2]

In [17]:
# locate folder with images
seattle_folder ="seattle_with_icons/"

In [18]:
append_img_url(seattle_curb_ramp, seattle_folder)

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,Unnamed: 14,Unnamed: 15,Unnamed: 16,geometry,cluster_id,img_url
10,Devon Snyder,19,482,University District,CurbRamp,47.661415,-122.320831,j3GefSTHZmrn2LHTGg8SFA,2022-05-10 14:08:11.854-07,t,3.0,f,not enough landing space,narrow,surface problem,NaN,NaN,POINT (-122.32083 47.66142),30,img/seattle_with_icons/label_id_19.jpg
12,Devon Snyder,21,482,University District,CurbRamp,47.661472,-122.320847,xx0-vgC0WHpTn2feGbrbUA,2022-05-10 14:09:32.271-07,t,2.0,f,not enough landing space,NaN,NaN,NaN,NaN,POINT (-122.32085 47.66147),29,img/seattle_with_icons/label_id_21.jpg
13,Devon Snyder,22,482,University District,CurbRamp,47.661488,-122.320854,qCkbp2lszdGrR5s3hdSiTQ,2022-05-10 14:09:52.251-07,t,2.0,f,not enough landing space,surface problem,NaN,NaN,NaN,POINT (-122.32085 47.66149),28,img/seattle_with_icons/label_id_22.jpg
15,Devon Snyder,27,483,University District,CurbRamp,47.661472,-122.320702,qCkbp2lszdGrR5s3hdSiTQ,2022-05-10 14:10:59.364-07,t,3.0,f,not level with street,NaN,NaN,NaN,NaN,POINT (-122.32070 47.66147),27,img/seattle_with_icons/label_id_27.jpg
28,Devon Snyder,41,486,University District,CurbRamp,47.661228,-122.319885,hr-qZOb3MdbSB1ZNBjsSIw,2022-05-10 14:18:55.439-07,t,2.0,f,surface problem,a bit far away even zoomed in but tried best to rate,NaN,NaN,NaN,POINT (-122.31989 47.66123),41,img/seattle_with_icons/label_id_41.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981,mikey,2453,811,Windermere,CurbRamp,47.672070,-122.264488,942Ur_vL2EhxfX2_o0xWqw,2022-07-06 13:39:55.694-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-122.26449 47.67207),4,img/seattle_with_icons/label_id_2453.jpg
1984,mikey,2456,811,Windermere,CurbRamp,47.672146,-122.264488,942Ur_vL2EhxfX2_o0xWqw,2022-07-06 13:40:38.753-07,NaN,5.0,f,surface problem,missing tactile warning,driveway can be used,NaN,NaN,POINT (-122.26449 47.67215),107,img/seattle_with_icons/label_id_2456.jpg
1985,mikey,2457,810,Windermere,CurbRamp,47.672043,-122.264664,gERMvp1tDlo3k2UJA7AM6w,2022-07-06 13:41:11.207-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-122.26466 47.67204),3,img/seattle_with_icons/label_id_2457.jpg
2015,mikey,2488,810,Windermere,CurbRamp,47.672230,-122.267761,09WtcNVwH9KSZ8B4Hqm3Bg,2022-07-06 14:11:48.515-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-122.26776 47.67223),2,img/seattle_with_icons/label_id_2488.jpg


In [19]:
append_img_url(seattle_problem, seattle_folder)

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,Unnamed: 14,Unnamed: 15,Unnamed: 16,geometry,cluster_id,img_url
0,Devon Snyder,3,480,University District,Obstacle,47.661041,-122.320709,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:36.787-07,t,3.0,f,pole,NaN,NaN,NaN,NaN,POINT (-122.32071 47.66104),238,img/seattle_with_icons/label_id_3.jpg
1,Devon Snyder,4,480,University District,Obstacle,47.661026,-122.320702,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:52.367-07,t,2.0,f,pole,NaN,NaN,NaN,NaN,POINT (-122.32070 47.66103),236,img/seattle_with_icons/label_id_4.jpg
2,Devon Snyder,5,480,University District,SurfaceProblem,47.661060,-122.320709,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:01:21.608-07,t,2.0,f,bumpy,NaN,NaN,NaN,NaN,POINT (-122.32071 47.66106),92,img/seattle_with_icons/label_id_5.jpg
4,Devon Snyder,9,480,University District,Obstacle,47.661098,-122.320663,O0VZ-MF6Nrmjhkb2nCRhCw,2022-05-10 14:02:29.636-07,t,3.0,f,sign,temporary due to construction,NaN,NaN,NaN,POINT (-122.32066 47.66110),235,img/seattle_with_icons/label_id_9.jpg
18,Devon Snyder,30,483,University District,Obstacle,47.661419,-122.320488,iI69DtsYap_Fjiz-RZ4i1A,2022-05-10 14:13:05.844-07,f,2.0,f,pole,NaN,NaN,NaN,NaN,POINT (-122.32049 47.66142),111,img/seattle_with_icons/label_id_30.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,mikey,2487,810,Windermere,Obstacle,47.672207,-122.267700,09WtcNVwH9KSZ8B4Hqm3Bg,2022-07-06 14:11:36.715-07,NaN,4.0,f,pole,NaN,NaN,NaN,NaN,POINT (-122.26770 47.67221),6,img/seattle_with_icons/label_id_2487.jpg
2017,mikey,2490,851,Windermere,SurfaceProblem,47.672043,-122.267998,09WtcNVwH9KSZ8B4Hqm3Bg,2022-07-06 14:12:12.695-07,NaN,1.0,f,NaN,pine needles,NaN,NaN,NaN,POINT (-122.26800 47.67204),5,img/seattle_with_icons/label_id_2490.jpg
2020,mikey,2493,810,Windermere,SurfaceProblem,47.672043,-122.267929,Z6BJD4bSdDiMxwMdls5oNw,2022-07-06 14:12:40.653-07,NaN,1.0,f,uneven/slanted,NaN,NaN,NaN,NaN,POINT (-122.26793 47.67204),4,img/seattle_with_icons/label_id_2493.jpg
2021,mikey,2494,716,Windermere,Obstacle,47.671928,-122.268097,sOyfUL3XT8ETE-9XfeaXvQ,2022-07-06 14:14:34.691-07,NaN,1.0,f,vegetation,NaN,NaN,NaN,NaN,POINT (-122.26810 47.67193),3,img/seattle_with_icons/label_id_2494.jpg


In [20]:
#get center location of the geodf
seattle.dissolve().centroid

0    POINT (-122.33162 47.61003)
dtype: geometry

In [21]:
seattle_problem

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,Unnamed: 14,Unnamed: 15,Unnamed: 16,geometry,cluster_id,img_url
0,Devon Snyder,3,480,University District,Obstacle,47.661041,-122.320709,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:36.787-07,t,3.0,f,pole,NaN,NaN,NaN,NaN,POINT (-122.32071 47.66104),238,img/seattle_with_icons/label_id_3.jpg
1,Devon Snyder,4,480,University District,Obstacle,47.661026,-122.320702,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:52.367-07,t,2.0,f,pole,NaN,NaN,NaN,NaN,POINT (-122.32070 47.66103),236,img/seattle_with_icons/label_id_4.jpg
2,Devon Snyder,5,480,University District,SurfaceProblem,47.661060,-122.320709,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:01:21.608-07,t,2.0,f,bumpy,NaN,NaN,NaN,NaN,POINT (-122.32071 47.66106),92,img/seattle_with_icons/label_id_5.jpg
4,Devon Snyder,9,480,University District,Obstacle,47.661098,-122.320663,O0VZ-MF6Nrmjhkb2nCRhCw,2022-05-10 14:02:29.636-07,t,3.0,f,sign,temporary due to construction,NaN,NaN,NaN,POINT (-122.32066 47.66110),235,img/seattle_with_icons/label_id_9.jpg
18,Devon Snyder,30,483,University District,Obstacle,47.661419,-122.320488,iI69DtsYap_Fjiz-RZ4i1A,2022-05-10 14:13:05.844-07,f,2.0,f,pole,NaN,NaN,NaN,NaN,POINT (-122.32049 47.66142),111,img/seattle_with_icons/label_id_30.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,mikey,2487,810,Windermere,Obstacle,47.672207,-122.267700,09WtcNVwH9KSZ8B4Hqm3Bg,2022-07-06 14:11:36.715-07,NaN,4.0,f,pole,NaN,NaN,NaN,NaN,POINT (-122.26770 47.67221),6,img/seattle_with_icons/label_id_2487.jpg
2017,mikey,2490,851,Windermere,SurfaceProblem,47.672043,-122.267998,09WtcNVwH9KSZ8B4Hqm3Bg,2022-07-06 14:12:12.695-07,NaN,1.0,f,NaN,pine needles,NaN,NaN,NaN,POINT (-122.26800 47.67204),5,img/seattle_with_icons/label_id_2490.jpg
2020,mikey,2493,810,Windermere,SurfaceProblem,47.672043,-122.267929,Z6BJD4bSdDiMxwMdls5oNw,2022-07-06 14:12:40.653-07,NaN,1.0,f,uneven/slanted,NaN,NaN,NaN,NaN,POINT (-122.26793 47.67204),4,img/seattle_with_icons/label_id_2493.jpg
2021,mikey,2494,716,Windermere,Obstacle,47.671928,-122.268097,sOyfUL3XT8ETE-9XfeaXvQ,2022-07-06 14:14:34.691-07,NaN,1.0,f,vegetation,NaN,NaN,NaN,NaN,POINT (-122.26810 47.67193),3,img/seattle_with_icons/label_id_2494.jpg


In [31]:
append_img_url(chicago_problem, chicago_folder)

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,Unnamed: 14,Unnamed: 15,Unnamed: 16,geometry,cluster_id,clustered,img_url
80,Devon Snyder,99,447,Skokie,SurfaceProblem,42.022999,-87.728149,2vMz67hXm2ZGNcKHbyoR8g,2022-05-10 14:59:05.467-07,t,1.0,f,grass,NaN,NaN,NaN,NaN,POINT (-87.72815 42.02300),65,False,img/chicago_with_icons/label_id_99.jpg
81,Devon Snyder,100,447,Skokie,SurfaceProblem,42.023033,-87.728149,2vMz67hXm2ZGNcKHbyoR8g,2022-05-10 14:59:10.286-07,t,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72815 42.02303),67,False,img/chicago_with_icons/label_id_100.jpg
83,Devon Snyder,103,447,Skokie,SurfaceProblem,42.023464,-87.728004,BMhU3MUjwcgFnO2ax-6TxQ,2022-05-10 15:01:27.068-07,t,2.0,f,cracks,grass,NaN,NaN,NaN,POINT (-87.72800 42.02346),94,False,img/chicago_with_icons/label_id_103.jpg
85,Devon Snyder,105,447,Skokie,SurfaceProblem,42.023663,-87.728111,spnkMkInfnfp6Se2o7-t9Q,2022-05-10 15:02:43.061-07,t,1.0,f,grass,cracks,NaN,NaN,NaN,POINT (-87.72811 42.02366),92,False,img/chicago_with_icons/label_id_105.jpg
86,Devon Snyder,106,447,Skokie,SurfaceProblem,42.024178,-87.728104,-YaXYeIUgy5a5bYJcFDMQQ,2022-05-10 15:03:40.542-07,t,1.0,f,cracks,grass,NaN,NaN,NaN,POINT (-87.72810 42.02418),1,True,img/chicago_with_icons/label_id_106.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1898,mikey,2368,437,Skokie,SurfaceProblem,42.023743,-87.728401,UUSs_oAVBV5cqnoLKsUnlQ,2022-07-05 12:25:19.838-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72840 42.02374),7,True,img/chicago_with_icons/label_id_2368.jpg
1899,mikey,2369,437,Skokie,SurfaceProblem,42.023582,-87.728416,Ue0KqJfGftuJWhdBCneCkQ,2022-07-05 12:25:31.351-07,NaN,1.0,f,uneven/slanted,NaN,NaN,NaN,NaN,POINT (-87.72842 42.02358),6,True,img/chicago_with_icons/label_id_2369.jpg
1908,mikey,2378,404,Skokie,SurfaceProblem,42.025799,-87.728088,jfd17OkETDGKKw-I1zK6Jw,2022-07-05 12:38:58.8-07,NaN,1.0,f,bumpy,NaN,NaN,NaN,NaN,POINT (-87.72809 42.02580),5,False,img/chicago_with_icons/label_id_2378.jpg
1910,mikey,2380,404,Skokie,Obstacle,42.025677,-87.728088,Y-iz2Xxh7q_fAX_UH6mmVA,2022-07-05 12:39:07.658-07,NaN,1.0,t,NaN,NaN,NaN,NaN,NaN,POINT (-87.72809 42.02568),3,False,img/chicago_with_icons/label_id_2380.jpg


In [32]:
chicago_problem

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,Unnamed: 14,Unnamed: 15,Unnamed: 16,geometry,cluster_id,clustered,img_url
80,Devon Snyder,99,447,Skokie,SurfaceProblem,42.022999,-87.728149,2vMz67hXm2ZGNcKHbyoR8g,2022-05-10 14:59:05.467-07,t,1.0,f,grass,NaN,NaN,NaN,NaN,POINT (-87.72815 42.02300),65,False,img/chicago_with_icons/label_id_99.jpg
81,Devon Snyder,100,447,Skokie,SurfaceProblem,42.023033,-87.728149,2vMz67hXm2ZGNcKHbyoR8g,2022-05-10 14:59:10.286-07,t,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72815 42.02303),67,False,img/chicago_with_icons/label_id_100.jpg
83,Devon Snyder,103,447,Skokie,SurfaceProblem,42.023464,-87.728004,BMhU3MUjwcgFnO2ax-6TxQ,2022-05-10 15:01:27.068-07,t,2.0,f,cracks,grass,NaN,NaN,NaN,POINT (-87.72800 42.02346),94,False,img/chicago_with_icons/label_id_103.jpg
85,Devon Snyder,105,447,Skokie,SurfaceProblem,42.023663,-87.728111,spnkMkInfnfp6Se2o7-t9Q,2022-05-10 15:02:43.061-07,t,1.0,f,grass,cracks,NaN,NaN,NaN,POINT (-87.72811 42.02366),92,False,img/chicago_with_icons/label_id_105.jpg
86,Devon Snyder,106,447,Skokie,SurfaceProblem,42.024178,-87.728104,-YaXYeIUgy5a5bYJcFDMQQ,2022-05-10 15:03:40.542-07,t,1.0,f,cracks,grass,NaN,NaN,NaN,POINT (-87.72810 42.02418),1,True,img/chicago_with_icons/label_id_106.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1898,mikey,2368,437,Skokie,SurfaceProblem,42.023743,-87.728401,UUSs_oAVBV5cqnoLKsUnlQ,2022-07-05 12:25:19.838-07,NaN,1.0,f,NaN,NaN,NaN,NaN,NaN,POINT (-87.72840 42.02374),7,True,img/chicago_with_icons/label_id_2368.jpg
1899,mikey,2369,437,Skokie,SurfaceProblem,42.023582,-87.728416,Ue0KqJfGftuJWhdBCneCkQ,2022-07-05 12:25:31.351-07,NaN,1.0,f,uneven/slanted,NaN,NaN,NaN,NaN,POINT (-87.72842 42.02358),6,True,img/chicago_with_icons/label_id_2369.jpg
1908,mikey,2378,404,Skokie,SurfaceProblem,42.025799,-87.728088,jfd17OkETDGKKw-I1zK6Jw,2022-07-05 12:38:58.8-07,NaN,1.0,f,bumpy,NaN,NaN,NaN,NaN,POINT (-87.72809 42.02580),5,False,img/chicago_with_icons/label_id_2378.jpg
1910,mikey,2380,404,Skokie,Obstacle,42.025677,-87.728088,Y-iz2Xxh7q_fAX_UH6mmVA,2022-07-05 12:39:07.658-07,NaN,1.0,t,NaN,NaN,NaN,NaN,NaN,POINT (-87.72809 42.02568),3,False,img/chicago_with_icons/label_id_2380.jpg


In [24]:
seattle_problem["clustered"] = seattle_problem.duplicated(subset="cluster_id", keep=False)

In [25]:
seattle_problem.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/seattle_problem_221109.json', driver="GeoJSON") 

In [26]:
seattle_curb_ramp["clustered"] = seattle_curb_ramp.duplicated(subset="cluster_id", keep=False)

In [27]:
seattle_curb_ramp.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/seattle_curb_ramp_221109.json', driver="GeoJSON") 

In [28]:
chicago_curb_ramp["clustered"] = chicago_curb_ramp.duplicated(subset="cluster_id", keep=False)

In [29]:
chicago_curb_ramp.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/chicago_curb_ramp_221109.json', driver="GeoJSON") 

In [33]:
chicago_problem["clustered"] = chicago_problem.duplicated(subset="cluster_id", keep=False)

In [34]:
chicago_problem.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/chicago_problem_221109.json', driver="GeoJSON") 